In [24]:
import pandas as pd
import datetime
data = pd.read_csv('test/test_presentation.txt')
data=data.drop(columns=['business_dt'])
data=data.drop_duplicates()
print(data)

cust_id             contr_id   draw_dt  draw_amt   loan_bal  \
0        jd_5c3b9fb704882  1180865269636026369  20191006     25000  16666.700   
1        jd_69d52b9a88249  1213466788482863105  20200104      1000   1000.000   
2        jd_4dafab5a9b4c9  1223443016899952641  20200201       625    625.000   
3              Samuel-Jay  1223065218117046272  20200131       625      0.000   
4        jd_4b6d43502b95a  1196620391963955201  20191119      1250   1041.625   
...                   ...                  ...       ...       ...        ...   
1109769  jd_5e4f21eef857d  1200990999543820289  20191201      3750      0.000   
1109793            Paul_波  1154365715302862848  20190725      1750    583.325   
1110253  331542887-716622  1201336517520310273  20191202     25000      0.000   
1110275   diyutianshi1314  1211888270812680193  20191231      6250      0.000   
1110320  jd_6cbcad941f6d2  1145574672231063553  20190701       625    416.700   

          prod_id  in_pay_amt  loan_term  loa

In [25]:
def func_term_count(data):
    data['to_term'] = data['contr_id'].count()
    return data
data=data.groupby('cust_id').apply(func_term_count)
data=data.drop_duplicates()
print(data)


cust_id             contr_id   draw_dt  draw_amt   loan_bal  \
0        jd_5c3b9fb704882  1180865269636026369  20191006     25000  16666.700   
1        jd_69d52b9a88249  1213466788482863105  20200104      1000   1000.000   
2        jd_4dafab5a9b4c9  1223443016899952641  20200201       625    625.000   
3              Samuel-Jay  1223065218117046272  20200131       625      0.000   
4        jd_4b6d43502b95a  1196620391963955201  20191119      1250   1041.625   
...                   ...                  ...       ...       ...        ...   
1109769  jd_5e4f21eef857d  1200990999543820289  20191201      3750      0.000   
1109793            Paul_波  1154365715302862848  20190725      1750    583.325   
1110253  331542887-716622  1201336517520310273  20191202     25000      0.000   
1110275   diyutianshi1314  1211888270812680193  20191231      6250      0.000   
1110320  jd_6cbcad941f6d2  1145574672231063553  20190701       625    416.700   

          prod_id  in_pay_amt  loan_term  loa

In [26]:

# # # #去重，下一个付息日
data=data.drop(columns=['next_pay_int_dt'])
data=data.drop(columns=[' loan_bal'])
data=data.drop(columns=['loan_term'])
data=data.drop_duplicates()
data=data.drop(columns=['mat_dt'])
data=data.drop(columns=['draw_dt'])
data=data.drop_duplicates()
print(data)

cust_id             contr_id  draw_amt   prod_id  \
0        jd_5c3b9fb704882  1180865269636026369     25000         3   
1        jd_69d52b9a88249  1213466788482863105      1000         3   
2        jd_4dafab5a9b4c9  1223443016899952641       625         3   
3              Samuel-Jay  1223065218117046272       625         3   
4        jd_4b6d43502b95a  1196620391963955201      1250         3   
...                   ...                  ...       ...       ...   
1109654  jd_52f8fdb6092fe  1178562587361558528      6250         3   
1109704  jd_6fd1e9d9a59d8  1140934622767484929      1750         3   
1109705  jd_52f8fdb6092fe  1178561639604101121       625         3   
1109793            Paul_波  1154365715302862848      1750         3   
1110320  jd_6cbcad941f6d2  1145574672231063553       625         3   

         in_pay_amt  loan_used_rate  out_pay_amt  outsheet_term  overdue  \
0             20000        0.000293            0              0        0   
1               800      

In [27]:
data['prin_overdue_count']=data.apply(lambda x: int('0') if x['prin_overdue_dt']=='\\N' else int('1'), axis=1)
data=data.drop(columns=['prin_repay_freq '])
def func_overdue_count(data):
    data['overdue_term'] =data['overdue_term']+data['prin_overdue_count'].sum()
    return data
data=data.groupby('contr_id').apply(func_overdue_count)

data['to_overdue_term']=data.apply(lambda x: int(x['outsheet_term'])+int(x['overdue_term']), axis=1)
data=data.drop(columns=['prin_overdue_dt'])
data=data.drop(columns=['outsheet_term'])
data=data.drop(columns=['overdue_term'])
data=data.drop_duplicates()
print(data)

cust_id             contr_id  draw_amt   prod_id  \
0        jd_5c3b9fb704882  1180865269636026369     25000         3   
1        jd_69d52b9a88249  1213466788482863105      1000         3   
2        jd_4dafab5a9b4c9  1223443016899952641       625         3   
3              Samuel-Jay  1223065218117046272       625         3   
4        jd_4b6d43502b95a  1196620391963955201      1250         3   
...                   ...                  ...       ...       ...   
1108878  jd_706a4769eb251  1188811595740905473       625         3   
1109233     15985791013_p  1166016032180273153      5875         3   
1109654  jd_52f8fdb6092fe  1178562587361558528      6250         3   
1109705  jd_52f8fdb6092fe  1178561639604101121       625         3   
1110320  jd_6cbcad941f6d2  1145574672231063553       625         3   

         in_pay_amt  loan_used_rate  out_pay_amt  overdue  overdue_grace  \
0             20000        0.000293            0        0              0   
1               800      

In [28]:
data['overdue_loan_bal']=data.apply(lambda x: float(x['overdue_loan_bal'])*(int(x['overdue'])*0.04+x['overdue_grace']*0.02+1), axis=1)
data=data.drop(columns=['overdue'])
data=data.drop(columns=['overdue_grace'])
data=data.drop_duplicates()
print(data)

cust_id             contr_id  draw_amt   prod_id  \
0        jd_5c3b9fb704882  1180865269636026369     25000         3   
1        jd_69d52b9a88249  1213466788482863105      1000         3   
2        jd_4dafab5a9b4c9  1223443016899952641       625         3   
3              Samuel-Jay  1223065218117046272       625         3   
4        jd_4b6d43502b95a  1196620391963955201      1250         3   
...                   ...                  ...       ...       ...   
1108878  jd_706a4769eb251  1188811595740905473       625         3   
1109233     15985791013_p  1166016032180273153      5875         3   
1109654  jd_52f8fdb6092fe  1178562587361558528      6250         3   
1109705  jd_52f8fdb6092fe  1178561639604101121       625         3   
1110320  jd_6cbcad941f6d2  1145574672231063553       625         3   

         in_pay_amt  loan_used_rate  out_pay_amt  overdue_loan_bal  to_term  \
0             20000        0.000293            0            0.0000       23   
1               800

In [29]:
def func_rate_mean(data):
    data['loan_used_rate'] = data['loan_used_rate'].mean()
    return data
data=data.groupby('contr_id').apply(func_rate_mean)
def func_in_pay_max(data):
    data['in_pay_amt'] = data['in_pay_amt'].max()
    return data
data=data.groupby('contr_id').apply(func_in_pay_max)
print(data)

cust_id             contr_id  draw_amt   prod_id  \
0        jd_5c3b9fb704882  1180865269636026369     25000         3   
1        jd_69d52b9a88249  1213466788482863105      1000         3   
2        jd_4dafab5a9b4c9  1223443016899952641       625         3   
3              Samuel-Jay  1223065218117046272       625         3   
4        jd_4b6d43502b95a  1196620391963955201      1250         3   
...                   ...                  ...       ...       ...   
1108878  jd_706a4769eb251  1188811595740905473       625         3   
1109233     15985791013_p  1166016032180273153      5875         3   
1109654  jd_52f8fdb6092fe  1178562587361558528      6250         3   
1109705  jd_52f8fdb6092fe  1178561639604101121       625         3   
1110320  jd_6cbcad941f6d2  1145574672231063553       625         3   

         in_pay_amt  loan_used_rate  out_pay_amt  overdue_loan_bal  to_term  \
0             20000        0.000293            0            0.0000       23   
1               800

In [30]:
def func_out_pay_max(data):
    data['out_pay_amt'] = data['out_pay_amt'].max()
    return data
data=data.groupby('contr_id').apply(func_out_pay_max)
def func_term_max(data):
    data['max_overdue_loan_bal'] = data['overdue_loan_bal'].max()
    return data
data=data.groupby('contr_id').apply(func_term_max)
data=data.drop(columns=['overdue_loan_bal'])
data=data.drop_duplicates()
print(data)

cust_id             contr_id  draw_amt   prod_id  \
0        jd_5c3b9fb704882  1180865269636026369     25000         3   
1        jd_69d52b9a88249  1213466788482863105      1000         3   
2        jd_4dafab5a9b4c9  1223443016899952641       625         3   
3              Samuel-Jay  1223065218117046272       625         3   
4        jd_4b6d43502b95a  1196620391963955201      1250         3   
...                   ...                  ...       ...       ...   
1096264  jd_7b1ae7e20f759  1182657512101388288      1250         3   
1101014  jd_6b471b74fa3d8  1197475249818480640      3750         3   
1101592  jd_4565501a6ebdc  1197290557857116160      6250         3   
1109654  jd_52f8fdb6092fe  1178562587361558528      6250         3   
1109705  jd_52f8fdb6092fe  1178561639604101121       625         3   

         in_pay_amt  loan_used_rate  out_pay_amt  to_term  prin_overdue_count  \
0             20000        0.000293            0       23                   0   
1              

In [31]:
data.to_csv('./test/handle_test_presentation.csv',index=False)
data = pd.read_csv('test/handle_test_presentation.csv')
def func_uverdue_term_max(data):
    data['to_overdue_term'] = data['to_overdue_term'].max()
    return data
data=data.groupby('contr_id').apply(func_uverdue_term_max)
data=data.drop(columns=['prin_overdue_count'])

def func_term_count(data):
    data['to_contr'] = data['contr_id'].count()
    return data
data=data.groupby('cust_id').apply(func_term_count)
data=data.drop(columns=['contr_id'])
data=data.drop_duplicates()

In [32]:
def func_rate_mean(data):
    data['loan_used_rate'] = data['loan_used_rate'].mean()
    return data
data=data.groupby('cust_id').apply(func_rate_mean)
def func_draw_amt_mean(data):
    data['draw_amt'] = data['draw_amt'].mean()
    return data
data=data.groupby('cust_id').apply(func_draw_amt_mean)
def func_in_pay_amt_mean(data):
    data['in_pay_amt'] = data['in_pay_amt'].mean()
    return data
data=data.groupby('cust_id').apply(func_in_pay_amt_mean)


In [33]:
def func_out_pay_amt_mean(data):
    data['out_pay_amt'] = data['out_pay_amt'].mean()
    return data
data=data.groupby('cust_id').apply(func_out_pay_amt_mean)
def func_term_sum(data):
    data['max_overdue_loan_bal'] = data['max_overdue_loan_bal'].sum()
    return data
data=data.groupby('cust_id').apply(func_term_sum)
def func_uverdue_term_sum(data):
    data['to_overdue_term'] = data['to_overdue_term'].sum()
    return data
data=data.groupby('cust_id').apply(func_uverdue_term_sum)
data=data.drop_duplicates()
print(data)
data.to_csv('./test/handle_test_presentation.csv',index=False)


cust_id     draw_amt   prod_id   in_pay_amt  loan_used_rate  \
0      jd_5c3b9fb704882  9900.000000         3  7920.000000        0.000312   
1      jd_69d52b9a88249  2666.666667         3  2133.333333        0.000500   
2      jd_4dafab5a9b4c9  1500.000000         3  1200.000000        0.000325   
3            Samuel-Jay  6105.769231         3  4884.615385        0.000350   
4      jd_4b6d43502b95a  5625.000000         3  4500.000000        0.000356   
...                 ...          ...       ...          ...             ...   
20439     bosaidong1100   625.000000         3   500.000000        0.000375   
20441     15830587521_p   625.000000         3   500.000000        0.000500   
20449  jd_524dcdcd90e99  5625.000000         3  4500.000000        0.000300   
20454  jd_6592bf06edf24  1250.000000         3  1000.000000        0.000300   
20458  jd_7c5196f829f87  3750.000000         3  3000.000000        0.000500   

       out_pay_amt  to_term  to_overdue_term  max_overdue_loan_bal 